<a href="https://colab.research.google.com/github/LithiraHettiarachchi/ForecastW/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
from scipy import stats
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

In [5]:
load_data = pd.read_csv('/content/drive/MyDrive/ForestFire_Tracker/forestfires.csv')

In [25]:
print(load_data)

     X  Y month  day  FFMC    DMC     DC   ISI  temp  RH  wind  rain   area
0    7  5   mar  fri  86.2   26.2   94.3   5.1   8.2  51   6.7   0.0   0.00
1    7  4   oct  tue  90.6   35.4  669.1   6.7  18.0  33   0.9   0.0   0.00
2    7  4   oct  sat  90.6   43.7  686.9   6.7  14.6  33   1.3   0.0   0.00
3    8  6   mar  fri  91.7   33.3   77.5   9.0   8.3  97   4.0   0.2   0.00
4    8  6   mar  sun  89.3   51.3  102.2   9.6  11.4  99   1.8   0.0   0.00
..  .. ..   ...  ...   ...    ...    ...   ...   ...  ..   ...   ...    ...
512  4  3   aug  sun  81.6   56.7  665.6   1.9  27.8  32   2.7   0.0   6.44
513  2  4   aug  sun  81.6   56.7  665.6   1.9  21.9  71   5.8   0.0  54.29
514  7  4   aug  sun  81.6   56.7  665.6   1.9  21.2  70   6.7   0.0  11.16
515  1  4   aug  sat  94.4  146.0  614.7  11.3  25.6  42   4.0   0.0   0.00
516  6  3   nov  tue  79.5    3.0  106.7   1.1  11.8  31   4.5   0.0   0.00

[517 rows x 13 columns]


In [24]:
load_data.isnull().sum()

X        0
Y        0
month    0
day      0
FFMC     0
DMC      0
DC       0
ISI      0
temp     0
RH       0
wind     0
rain     0
area     0
dtype: int64

In [26]:
z_scores = np.abs(stats.zscore(load_data.iloc[: ,4:12]))
threshold = 3
outliers = np.where(z_scores > threshold)

# Remove outliers
data = load_data[(z_scores < threshold).all(axis=1)]


In [27]:
#Scaling the dataset
#Encoding data
label_encoder = LabelEncoder()
data['month'] = label_encoder.fit_transform(data['month'])

label_encoder = LabelEncoder()
data['day'] = label_encoder.fit_transform(data['day'])

# Print the encoded DataFrame
print(data)

     X  Y  month  day  FFMC    DMC     DC   ISI  temp  RH  wind  rain   area
0    7  5      7    0  86.2   26.2   94.3   5.1   8.2  51   6.7   0.0   0.00
1    7  4     10    5  90.6   35.4  669.1   6.7  18.0  33   0.9   0.0   0.00
2    7  4     10    2  90.6   43.7  686.9   6.7  14.6  33   1.3   0.0   0.00
5    8  6      1    3  92.3   85.3  488.0  14.7  22.2  29   5.4   0.0   0.00
6    8  6      1    1  92.3   88.9  495.6   8.5  24.1  27   3.1   0.0   0.00
..  .. ..    ...  ...   ...    ...    ...   ...   ...  ..   ...   ...    ...
512  4  3      1    3  81.6   56.7  665.6   1.9  27.8  32   2.7   0.0   6.44
513  2  4      1    3  81.6   56.7  665.6   1.9  21.9  71   5.8   0.0  54.29
514  7  4      1    3  81.6   56.7  665.6   1.9  21.2  70   6.7   0.0  11.16
515  1  4      1    2  94.4  146.0  614.7  11.3  25.6  42   4.0   0.0   0.00
516  6  3      9    5  79.5    3.0  106.7   1.1  11.8  31   4.5   0.0   0.00

[497 rows x 13 columns]


<ipython-input-27-501ceda16e37>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['month'] = label_encoder.fit_transform(data['month'])
<ipython-input-27-501ceda16e37>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['day'] = label_encoder.fit_transform(data['day'])


In [32]:

columns_to_normalize = ['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area']
scaler = MinMaxScaler()
normalized_dataset = data.copy()
normalized_dataset[columns_to_normalize] = scaler.fit_transform(normalized_dataset[columns_to_normalize])
print(normalized_dataset)

     X  Y  month  day      FFMC       DMC        DC       ISI      temp  \
0    7  5      7    0  0.526066  0.080472  0.101325  0.186047  0.192926   
1    7  4     10    5  0.734597  0.112383  0.775419  0.260465  0.508039   
2    7  4     10    2  0.734597  0.141172  0.796294  0.260465  0.398714   
5    8  6      1    3  0.815166  0.285467  0.563035  0.632558  0.643087   
6    8  6      1    1  0.815166  0.297954  0.571948  0.344186  0.704180   
..  .. ..    ...  ...       ...       ...       ...       ...       ...   
512  4  3      1    3  0.308057  0.186264  0.771315  0.037209  0.823151   
513  2  4      1    3  0.308057  0.186264  0.771315  0.037209  0.633441   
514  7  4      1    3  0.308057  0.186264  0.771315  0.037209  0.610932   
515  1  4      1    2  0.914692  0.496011  0.711622  0.474419  0.752412   
516  6  3      9    5  0.208531  0.000000  0.115867  0.000000  0.308682   

           RH      wind  rain      area  
0    0.480000  0.741176   0.0  0.000000  
1    0.240000  

In [33]:
X = normalized_dataset.drop('area', axis=1)
y = normalized_dataset['area']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
model = LinearRegression()


In [35]:
model.fit(X_train, y_train)

LinearRegression()